<a href="https://colab.research.google.com/github/namwootree/Portfolio/blob/main/Personal/%ED%8C%94%EB%8B%B9%EB%8C%90%20%ED%99%8D%EC%88%98%20%EC%95%88%EC%A0%84%EC%9A%B4%EC%98%81%EC%97%90%20%EB%94%B0%EB%A5%B8%20%ED%95%9C%EA%B0%95%20%EC%88%98%EC%9C%84%EC%98%88%EC%B8%A1%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from scipy import interpolate

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D

import os

In [4]:
# GPU 자원이 부족한 경우 아래 코드를 이용하세요
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
w_list = sorted(glob("/content/drive/MyDrive/시계열/팔당댐/water_data/*.csv"))
w_list

['/content/drive/MyDrive/시계열/팔당댐/water_data/data_2012.csv',
 '/content/drive/MyDrive/시계열/팔당댐/water_data/data_2013.csv',
 '/content/drive/MyDrive/시계열/팔당댐/water_data/data_2014.csv',
 '/content/drive/MyDrive/시계열/팔당댐/water_data/data_2015.csv',
 '/content/drive/MyDrive/시계열/팔당댐/water_data/data_2016.csv',
 '/content/drive/MyDrive/시계열/팔당댐/water_data/data_2017.csv',
 '/content/drive/MyDrive/시계열/팔당댐/water_data/data_2018.csv',
 '/content/drive/MyDrive/시계열/팔당댐/water_data/data_2019.csv',
 '/content/drive/MyDrive/시계열/팔당댐/water_data/data_2020.csv',
 '/content/drive/MyDrive/시계열/팔당댐/water_data/data_2021.csv',
 '/content/drive/MyDrive/시계열/팔당댐/water_data/data_2022.csv']

In [6]:
pd.read_csv(w_list[0]).shape

(26496, 15)

In [7]:
pd.read_csv(w_list[0]).head(4)

,ymdhm,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
0,2012-05-01 00:00,24.800,555.0,219.07,24.93,555.0,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18
1,2012-05-01 00:10,24.794,464.6,218.86,25.15,562.9,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18
2,2012-05-01 00:20,24.789,478.1,218.69,25.31,576.4,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18
3,2012-05-01 00:30,24.789,464.8,218.69,25.31,563.1,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17


In [8]:
train_data = []
train_label = []
num = 0

for i in w_list[:-1]:
    
    tmp = pd.read_csv(i)
    tmp = tmp.replace(" ", np.nan)
    tmp = tmp.interpolate(method = 'cubic')
    tmp = tmp.fillna(method = 'pad')
    
    for j in tqdm(range(len(tmp)-432)):
        train_data.append(np.array(tmp.loc[j:j + 431, ["inf",
                                                       "tototf",
                                                       "fw_1018662",
                                                       "fw_1018683", "fw_1019630"]]).astype(float))
        
        train_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                      "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 26064/26064 [00:35<00:00, 736.76it/s]


In [9]:
train_data = np.array(train_data)
train_label = np.array(train_label)

print(train_data.shape)
print(train_label.shape)

(260640, 432, 5)
(260640, 4)


In [10]:
test_data = []
test_label = []

tmp = pd.read_csv(w_list[-1])
tmp = tmp.replace(" ", np.nan)
# 이전값을 사용
tmp = tmp.fillna(method = 'pad')
tmp = tmp.fillna(0)
    
#tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]] = tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]]*100
    
for j in tqdm(range(4032, len(tmp)-432)):
    test_data.append(np.array(tmp.loc[j:j + 431, ["inf",
                                                  "tototf",
                                                  "fw_1018662",
                                                  "fw_1018683", "fw_1019630"]]).astype(float))
        
    test_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                    "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 6912/6912 [00:09<00:00, 736.13it/s]


In [11]:
test_data = np.array(test_data)
test_label = np.array(test_label)

print(test_data.shape)
print(test_label.shape)

(6912, 432, 5)
(6912, 4)


In [12]:
# Deep Neural Network
from tensorflow.keras import layers, models
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.models import Sequential
import time

In [13]:
from tensorflow.keras.metrics import RootMeanSquaredError

In [14]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

In [23]:
tf.keras.initializers.Initializer()


input_shape = (train_data[0].shape[0], train_data[0].shape[1])

model = Sequential()
model.add(GRU(64, input_shape=input_shape, return_sequences=True))
model.add(GRU(64, dropout=0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation = 'relu'))


In [24]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)

In [25]:
model.compile(optimizer = optimizer, loss = root_mean_squared_error, metrics=[RootMeanSquaredError()])

In [ ]:
model.fit(train_data, train_label, epochs=20, batch_size=2048, validation_split=0.1)

Epoch 1/20
 69/115 [=================>............] - ETA: 2:27 - loss: 299.0682 - root_mean_squared_error: 307.2026

In [ ]:
pred = model.predict(test_data)
pred = pd.DataFrame(pred)

In [ ]:
sample_submission = pd.read_csv("/content/drive/MyDrive/시계열/팔당댐/sample_submission.csv")

sample_submission["wl_1018662"] = pred[0]
sample_submission["wl_1018680"] = pred[1]
sample_submission["wl_1018683"] = pred[2]
sample_submission["wl_1019630"] = pred[3]

In [ ]:
sample_submission.to_csv("GRU_FL.csv", index = False)